# 📓 The GenAI Revolution Cookbook

**Title:** How to Build Explainability AI by Design with OPA, MCP, and Neo4j

**Description:** Build production-ready control layers for agent systems using OPA, MCP, and Neo4j. Enforce policies, capture auditable traces, and insert human approvals without slowing teams or shipping velocity.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



You want an agent that is smart, not reckless. You also want a security boundary that the agent cannot talk around. This project shows you how to combine CrewAI, an MCP server, OPA, and a small ML PII classifier to enforce tool access on ingress and prevent sensitive data from leaking on egress. You will see how each decision is audited. You will also see how policy stays outside the LLM so you can change rules without touching prompts or model weights.

## 0\. What this system guarantees

1. Agents cannot bypass policy. The agent never talks to tools directly. Every call flows through an MCP server that consults OPA before any action.
2. Tool access rules live only in OPA. You manage permissions in one place. You do not spread them across agents, tools, or code paths.
3. PII cannot leak back to the agent. The MCP server classifies content with a small ML model, then enforces OPA egress rules that redact or block as needed.
4. All decisions are auditable. Every check writes a structured record so you can explain and replay decisions later.
5. ML classifies data, OPA decides policy. The model produces signals. OPA turns those signals into allow, redact, or block.

## 1\. Project structure

You will work with a small, clear layout. Each folder has one responsibility. Policies and data live under opa. The MCP server contains the enforcement logic. CrewAI connects through a single tool integration.

In [ ]:
project/
├── opa/
│   ├── policies/
│   │   ├── tool_access.rego
│   │   └── egress_control.rego
│   └── data/
│       └── tool_catalog.json
├── audit.py
├── approvals.py
├── opa_client.py
├── pii_classifier.py
├── tools.py
├── mcp_server.py
├── crew_tools.py
└── crew.py

## 2\. Install dependencies

Set up your environment and install what you need for CrewAI, MCP, OPA, and spaCy. This gives you the runtime for policy checks, content classification, and agent execution.

In [ ]:
pip install crewai fastapi uvicorn requests neo4j spacy
python -m spacy download en_core_web_sm

Run OPA:

In [ ]:
docker run --rm -p 8181:8181 \
  -v "$PWD/opa":/opa \
  openpolicyagent/opa:latest \
  run --server /opa/policies /opa/data

Tip. Keep OPA running in a separate terminal. You will want to see policy decision logs as you test.

## 3\. Tool catalog (FACTS ONLY)

This file declares the tools that exist, their inputs and outputs, and any static metadata. It does not contain permissions or rules. It is a source of truth for capabilities, not policy.

opa/data/tool\_catalog.json

```json
{
  "tools": {
    "initiate_refund": {
      "risk_level": "high",
      "data_classification": "financial",
      "max_auto_amount": 500,
      "approval_threshold": 2000
    }
  }
}
```

No permissions.
No agents.
No rules.

Practical note. You can add new tools and attributes here. OPA will use these facts when it evaluates ingress policy. This keeps policy logic clean, since it can reference the catalog instead of parsing code.

## 4\. OPA ingress policy (tool access)

This policy decides whether the agent can call a tool, given who is asking and for what purpose. The MCP server sends OPA the agent identity, tool name, and request context. OPA responds with allow or deny, with reasons. You can add conditions like time, environment, or purpose.

opa/policies/tool\_access.rego

```rego
package tool_access

default decision = "deny"

tool := data.tools[input.tool.id]

decision = "allow" {
  input.agent.role == "support_agent"
  input.arguments.amount <= tool.max_auto_amount
}

decision = "require_approval" {
  input.agent.role == "support_agent"
  input.arguments.amount > tool.max_auto_amount
  input.arguments.amount <= tool.approval_threshold
}
```

When you test, try changing the purpose or agent role in the request. You will see OPA flip decisions without any LLM prompt changes.

## 5\. OPA egress policy (PII control)

This policy looks at the content that is about to leave the MCP server. The small ML classifier flags PII and assigns categories and confidence. OPA reads those signals and decides whether to allow, redact, or block. You get central control over how strict you want to be.

opa/policies/egress\_control.rego

```rego
package egress_control

default action = "block"

tool := data.tools[input.tool.id]

# No PII → OK
action = "allow" {
  not input.egress.contains_pii
}

# PII present, agent not cleared → redact
action = "redact" {
  input.egress.contains_pii
  input.agent.clearance != "pii_allowed"
}

# PII present, agent cleared → allow
action = "allow" {
  input.egress.contains_pii
  input.agent.clearance == "pii_allowed"
}
```

Practical paths you can try:

* Allow with redaction. Let the response through but mask PII fields. Good for customer support summaries.
* Block with human\-in\-the\-loop. Stop the response and request approval. Useful for edge cases where high confidence PII appears.
* Allow as\-is for low risk data. Keep latency low where there is no sensitive content.

## 6\. Audit logger

Every ingress and egress decision creates a structured audit entry. You can correlate requests with a trace or request ID, see the inputs that drove the decision, and read the exact rule path that fired. This gives you explainability for security reviews and incident response.

audit.py

In [ ]:
import json
import uuid
from datetime import datetime

def audit_log(event: dict) -> str:
    event["event_id"] = event.get("event_id") or str(uuid.uuid4())
    event["timestamp"] = datetime.utcnow().isoformat()
    with open("audit.log", "a", encoding="utf-8") as f:
        f.write(json.dumps(event, ensure_ascii=False) + "\n")
    return event["event_id"]

When something is denied or redacted, check the audit log. You will see the model signals, the OPA input, and the decision details.

## 7\. Human approval store

Some decisions require a person to approve. This store keeps track of pending approvals, who approved, and when. It also gives you a simple way to expire approvals and prevent reuse.

approvals.py

In [ ]:
import uuid

PENDING_APPROVALS = {}

def create_approval(facts: dict) -> str:
    token = str(uuid.uuid4())
    PENDING_APPROVALS[token] = facts
    return token

def consume_approval(token: str):
    return PENDING_APPROVALS.pop(token, None)

Try a flow where egress is blocked pending approval. Approve it, then resend the same request. The MCP server will record a different outcome with a clean audit trail.

## 8\. OPA client

This client wraps calls to OPA. The MCP server uses it to ask OPA for ingress and egress decisions with a consistent input schema. You can swap OPA endpoints or namespaces without changing the rest of your system.

opa\_client.py

In [ ]:
import requests

OPA = "http://localhost:8181/v1/data"

def tool_access_decision(facts: dict) -> str:
    r = requests.post(f"{OPA}/tool_access/decision", json={"input": facts}, timeout=3)
    return r.json()["result"]

def egress_action(facts: dict) -> str:
    r = requests.post(f"{OPA}/egress_control/action", json={"input": facts}, timeout=3)
    return r.json()["result"]

If a policy query fails, the client should fail closed. In practice that means deny access or block egress. The audit log will capture the error and the fail\-closed decision.

## 9\. Small ML PII classifier (NO regex)

The classifier uses a compact spaCy pipeline to detect entities like names, emails, phone numbers, and IDs. This model does not rely on regex. It gives you probabilistic signals that travel with the content to OPA.

pii\_classifier.py

In [ ]:
import json
import spacy

nlp = spacy.load("en_core_web_sm")

LABEL_MAP = {
    "PERSON": "person_name",
    "GPE": "location",
    "LOC": "location",
    "ORG": "organization",
    "DATE": "date"
}

def classify_pii(result):
    text = result if isinstance(result, str) else json.dumps(result, ensure_ascii=False)
    doc = nlp(text)

    findings = []
    for ent in doc.ents:
        if ent.label_ in LABEL_MAP:
            findings.append({
                "type": LABEL_MAP[ent.label_],
                "text": ent.text
            })

    return {
        "contains_pii": len(findings) > 0,
        "pii_types": sorted({f["type"] for f in findings}),
        "findings": findings
    }

You can tune thresholds to trade off between false positives and false negatives. You can also map model labels to your own sensitivity levels. These become the inputs that OPA understands.

## 10\. Tool implementation (returns PII internally)

The tool can return raw PII internally. That is expected. The key is that the agent never receives this raw data. The MCP server inspects the tool result, classifies it, and then applies OPA egress policy before anything leaves the boundary.

tools.py

In [ ]:
def initiate_refund(customer_id: str, amount: int):
    return {
        "status": "executed",
        "customer_id": customer_id,
        "message": (
            f"Refund of ${amount} issued to customer {customer_id}. "
            f"Contact: John Doe, Montreal."
        )
    }

This pattern lets you integrate real systems like CRMs or support databases without trusting the agent to police itself.

## 11\. MCP server (the enforcement boundary)

The MCP server sits between the agent and your tools. It is the gatekeeper for every call. It makes two policy checks and captures a full audit.

mcp\_server.py

In [ ]:
from fastapi import FastAPI
from audit import audit_log
from approvals import create_approval, consume_approval
from opa_client import tool_access_decision, egress_action
from pii_classifier import classify_pii
from tools import initiate_refund

app = FastAPI()

def execute_tool(name, args):
    if name == "initiate_refund":
        return initiate_refund(**args)
    raise ValueError("Unknown tool")

@app.post("/mcp/tool/{tool_name}")
def call_tool(tool_name: str, payload: dict):
    facts = {
        "agent": payload["agent"],
        "tool": {"id": tool_name},
        "arguments": payload["arguments"],
        "environment": payload.get("environment", "production")
    }

    decision = tool_access_decision(facts)
    event_id = audit_log({
        "phase": "access",
        "agent": facts["agent"],
        "tool": tool_name,
        "arguments": facts["arguments"],
        "decision": decision
    })
    facts["event_id"] = event_id

    if decision == "deny":
        return {"error": "Denied by policy"}

    if decision == "require_approval":
        token = create_approval(facts)
        audit_log({"event_id": event_id, "phase": "approval_requested", "token": token})
        return {"status": "pending_approval", "approval_token": token}

    raw = execute_tool(tool_name, facts["arguments"])
    egress = classify_pii(raw)
    facts["egress"] = egress

    action = egress_action(facts)
    audit_log({
        "event_id": event_id,
        "phase": "egress",
        "action": action,
        "egress": egress
    })

    if action == "allow":
        return raw

    if action == "redact":
        return {"status": "redacted", "pii_types": egress["pii_types"]}

    return {"error": "Blocked by egress policy"}

@app.post("/mcp/approve/{token}")
def approve(token: str):
    facts = consume_approval(token)
    if not facts:
        return {"error": "Invalid token"}
    raw = execute_tool(facts["tool"]["id"], facts["arguments"])
    return raw

Run it:

In [ ]:
uvicorn mcp_server:app --port 3333

What happens for each request:

1. Agent asks the MCP server to use a tool. The server calls OPA to check ingress. If denied, the agent gets a safe explanation. The audit log records the denial.
2. If allowed, the server runs the tool and keeps the raw result inside the boundary.
3. The server runs the PII classifier on the result and sends signals to OPA for egress. OPA returns allow, redact, or block, with reasons.
4. The server applies the decision, optionally requests human approval, and returns only safe output to the agent.
5. The server writes a complete audit entry with inputs, decisions, and outcomes.

## 12\. CrewAI tool (calls MCP)

From the agent’s perspective there is a single tool. It talks only to the MCP server. The agent does not know about OPA, internal tools, or your data sources. You keep the enforcement surface small and consistent.

crew\_tools.py

In [ ]:
import requests
from crewai.tools import tool

MCP = "http://localhost:3333/mcp/tool/initiate_refund"

@tool("initiate_refund")
def initiate_refund_tool(customer_id: str, amount: int):
    payload = {
        "agent": {
            "id": "agent-1",
            "role": "support_agent",
            "clearance": "no_pii"
        },
        "arguments": {
            "customer_id": customer_id,
            "amount": amount
        }
    }
    return requests.post(MCP, json=payload).json()

If you add new internal tools, do it behind the MCP server. The agent integration does not change.

## 13\. CrewAI agent

This is a simple agent that uses the MCP\-backed tool. You can prompt it to fetch or summarize a record. It will reason about what to ask. It will not see PII unless policy allows it.

crew.py

In [ ]:
from crewai import Agent, Task, Crew
from crew_tools import initiate_refund_tool

agent = Agent(
    role="support_agent",
    goal="Handle refunds safely",
    tools=[initiate_refund_tool],
    verbose=True
)

task = Task(
    description="Refund customer 123 for $250",
    agent=agent
)

crew = Crew(agents=[agent], tasks=[task])
print(crew.kickoff())

Try a prompt that asks for a customer profile. Watch how the agent gets a redacted summary instead of raw details. Then adjust OPA policy and rerun to see different outcomes without touching the prompt.

## 14\. What you will see

* Tool executes internally. The agent never touches the tool directly.
* PII detected by ML. The classifier tags entities and passes signals forward.
* OPA egress policy fires. You get allow, redact, or block, with reasons.
* Agent receives redacted response. Sensitive values are masked or removed.
* Full audit trail written. Every decision has a timestamp, inputs, and rule references.

## Final mental model

| Component | Responsibility |
| --- | --- |
| CrewAI | Reasoning |
| MCP server | Enforcement |
| spaCy SML | Classification |
| OPA | Policy decisions |
| Audit log | Explainability |

## Final one\-sentence summary

This system enforces ingress and egress policies outside the LLM using OPA, classifies sensitive data with a small ML model, prevents PII leakage, and records every decision for audit and explainability.